In [24]:
import datetime
import pymongo
import random
import uuid
from pymongo import MongoClient

In [25]:
N_BOOKMARKS = 5 # number of bookmarks

In [26]:
client = MongoClient('mongodb', 27017)

In [27]:
# Getting a Database
db = client.research_db

In [28]:
# List of bookmarks
# Scheme:
# document = 
# {user_id: 
#     [movie_id_1, movie_id_2]
# }
bookmarks = db.bookmarks

In [29]:
# Loading data

for i in range(N_BOOKMARKS):
    user_id = str(uuid.uuid4())
    movie_id = str(uuid.uuid4())
    bookmarks.insert_one(
        {"id": i,
         "user_id": user_id,
         "movie_id": [movie_id]
        }
    )


In [30]:
# Reading data
start = datetime.datetime.now()
for i in range(N_BOOKMARKS):
    id = random.randint(0, 15000)
    bookmarks.find_one({"id": id})
end = datetime.datetime.now()
print(end-start)

0:00:00.009678


In [ ]:
# Likes - like is a score
# Scheme:
# documents = 
# [{movie_id, user_id, score}]
